# జంతువుల నిపుణుల వ్యవస్థను అమలు చేయడం

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) నుండి ఒక ఉదాహరణ.

ఈ నమూనాలో, కొన్ని శారీరక లక్షణాల ఆధారంగా జంతువును నిర్ణయించే సులభమైన జ్ఞానాధారిత వ్యవస్థను అమలు చేస్తాము. ఈ వ్యవస్థను కింద ఉన్న AND-OR చెట్టు ద్వారా ప్రతిబింబించవచ్చు (ఇది మొత్తం చెట్టు భాగమే, మనం సులభంగా మరిన్ని నియమాలను జోడించవచ్చు):

![](../../../../../../translated_images/te/AND-OR-Tree.5592d2c70187f283.webp)


## మన స్వంత నిపుణుల వ్యవస్థ షెల్ బ్యాక్వర్డ్ ఇన్ఫెరెన్స్‌తో

ఉత్పత్తి నియమాలపై ఆధారపడి జ్ఞాన ప్రాతినిధ్యం కోసం ఒక సరళ భాషను నిర్వచించడానికి ప్రయత్నిద్దాం. నియమాలను నిర్వచించడానికి కీవర్డ్లుగా Python తరగతులను ఉపయోగిస్తాము. ప్రధానంగా 3 రకాల తరగతులు ఉంటాయి:
* `Ask` ఉపయోగించబడేది వినియోగదారునికి అడగాల్సిన ప్రశ్నను సూచించడానికి. ఇందులో సాధ్యమైన సమాధానాల సమాహారం ఉంటుంది.
* `If` ఒక నియమాన్ని సూచిస్తుంది, ఇది ఆ నియమపు విషయాంశాన్ని నిల్వచేసే సింటాక్టిక్ షుగర్ మాత్రమే
* `AND`/`OR` చూడండి, ఇవి పామును AND/OR శాఖలను సూచించడానికి తరగతులు. ఇవి కేవలం లోపల ఆర్గ్యుమెంట్ల జాబితాను నిల్వచేస్తాయి. కోడ్ సులభతరం చేసేందుకు, అన్నీ ఫంక్షనాలిటీ మాతృ తరగతి `Content` లో నిర్వచించబడతాయి.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

మన వ్యవస్థలో, పని జ్ఞాపకం **ఫ్యాక్ట్స్** జాబితాను **గుణము-విలువ జంటలుగా** కలిగి ఉంటుంది. జ్ఞానాధారం అనేది ఒక పెద్ద నిఘంటువు అని నిర్వచించవచ్చు, ఇది చర్యలు (పని జ్ఞాపకంలో చేర్చాల్సిన కొత్త ఫ్యాక్ట్స్) ను షరతులతో మ్యాప్ చేస్తుంది, అవి AND-OR వ్యక్తీకరణల రూపంలో ఉంటాయి. అలాగే, కొన్ని ఫ్యాక్ట్స్ ని `Ask` చేయవచ్చు.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

వెనుకబడిన భావనను నిర్వహించడానికి, మనం `Knowledgebase` తరగతిని నిర్వచించబోతున్నాము. ఇది కలిగి ఉంటుంది:
* పని `memory` - లక్షణాలను విలువలకు మ్యాప్ చేసే డిక్షనరీ
* పై పేర్కొన్న ఫార్మాట్‌లో ఉన్న Knowledgebase `rules`

రెండు ప్రధాన విధానాలు:
* విలువను పొందడానికి `get`, అవసరమైతే భావనను నడుపుతుంది. ఉదాహరణకు, `get('color')` రంగు స్థానం విలువను పొందుతుంది (అవసరమైతే అడుగుతుంది, మరియు తర్వాత ఉపయోగానికి పని మెమరీలో విలువను నిల్వ చేస్తుంది). మనం `get('color:blue')` అడిగితే, అది రంగును అడుగుతుంది, తరువాత రంగుపై ఆధారంగా `y`/`n` విలువను ఇస్తుంది.
* `eval` వాస్తవ భావనను నిర్వహిస్తుంది, అంటే AND/OR వృక్షాన్ని అన్వేషించి, ఉప-గోల్స్‌ను మూల్యాంకనం చేస్తుంది, మరియు ఇతర చర్యలు పొందుతుంది.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

ఇప్పుడు మన జంతు జ్ఞానాధారాన్ని నిర్వచించి సంప్రదింపును జరపుదాం. ఈ కాల్ మీకు కొన్ని ప్రశ్నలు అడగనుంది. అవును-కాదు ప్రశ్నలకు మీరు `y`/`n` టైప్ చేసి సమాధానం ఇవ్వవచ్చు, లేదా విస్తృత ప్రమేయాలున్న ప్రశ్నల కోసం సంఖ్య (0..N) సూచించవచ్చు.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## ఫార్వర్డ్ ఇన్ఫరెన్స్ కోసం Experta ఉపయోగించడం

తదుపరి ఉదాహరణలో, జ్ఞాన ప్రాతినిధ్యం కోసం లైబ్రరీలలో ఒకటైన [Experta](https://github.com/nilp0inter/experta) ఉపయోగించి ఫార్వర్డ్ ఇన్ఫరెన్స్‌ను అమలు చేయడానికి ప్రయత్నిస్తాము. **Experta** అనేది Python లో ఫార్వర్డ్ ఇన్ఫరెన్స్ సిస్టమ్లను సృష్టించడానికి ఉపయోగించే లైబ్రరీ, ఇది సాంప్రదాయాత్మక పాత సిస్టమ్ [CLIPS](http://www.clipsrules.net/index.html) కు చాలా సమానంగా ఉండేలా రూపొందించబడింది.

మేము కూడా ఫార్వర్డ్ చైనింగ్‌ను స్వయంగా ఏమీ పెద్ద తంత్రములు లేకుండా అమలు చేయవచ్చు, కానీ స్థానిక అమలు సాధారాణంగా చాలా సమర్థవంతంగా ఉండవు. మరింత ప్రభావవంతమైన నియమాలు సరిపోయే కొరకు ప్రత్యేక అల్గోరిథమ్ [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) ఉపయోగించబడుతుంది.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

మేము మా సిస్టమ్ను `KnowledgeEngine` ను subclass చేసే క్లాస్ గా నిర్వచిస్తాము. ప్రతి Rule ఒక విభిన్న ఫంక్షన్ ద్వారా నిర్వచించబడుతుంది, దీనికి `@Rule` అనోటేషన్ ఉంటుంది, ఇది ఆ Rule ఎప్పుడు అమలు చేయాలి అని సూచిస్తుంది. ఆ Rule లో, మేము `declare` ఫంక్షన్ ఉపయోగించి కొత్త వాస్తవాలను జత చేయవచ్చు, మరియు ఆ వాస్తవాలను జతచేయడం వల్ల ముందుకు inferencing ఇంజన్ మరికొన్ని Rules ను పిలుస్తుంది.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

ఒక్కసారి మనం ఒక జ్ఞానాధారాన్ని నిర్వచించిన తర్వాత, మనం మా పని జ్ఞాపకాన్ని కొన్ని ప్రారంభ వాస్తవాలతో నింపుతాము, మరియు ఆపై inferencing చేయడానికి `run()` మెటోడ్ను పిలుస్తాము. మీరు ఫలితంగా చూసుకోవచ్చు కొత్త infer చేయబడిన వాస్తవాలు పని జ్ఞాపకానికి జోడించబడ్డాయి, చివరి వాస్తవం ప్రాణి గురించి (మనం అన్ని ప్రారంభ వాస్తవాలను సరిగ్గా సెటప్ చేస్తే).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత సూచన**:
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తూనే ఉన్నప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో తప్పులు లేదా అసత్యతలు ఉండవచ్చని గమనించగలరు. అసలు పత్రం దాని స్వదేశ భాషలో ఉన్నదాన్ని అధికారిక మూలంగా పరిగణించాలి. కీలకమైన సమాచారానికి, వృత్తిపరమైన మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదం ఉపయోగంతో ఏర్పడే ఏదైనా అపవాదాలు లేదా తప్పుదారులపై మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
